In [82]:
import warnings
warnings.filterwarnings('ignore')

In [83]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# GM

In [3]:
#you can also query like this
blaze_gm_data = pd.read_sql_query("""
with base as
(
   select
      *,
      last_day(week_start) year_month,
      retail_value / quantity_sold per_unit_price,
      cogs / quantity_sold per_unit_cost,
      sales.retail_value - sales.total_discount + sales.total_tax sales_post_tax_post_discount,
      sales.retail_value - sales.total_discount sales_pre_tax_post_discount,
      sales.retail_value sales_pre_tax_pre_discount,
   from
      blaze_data.sales sales
      -- EXCLUDE SAMPLES,DISPLAY,PROMO and test
   where
      lower(product_category) not like '%sample%'
      and lower(product_category) not like '%display%'
      and lower(product_category) not like '%promo%'
      and lower(product_category) not like '%test%'
    and state = 'MA'
)
,
monthly_location_rev_and_cogs as
(
   select
      year_month,
      shop_id shopid,
      company_id companyid,
      sum(sales_post_tax_post_discount) rev_post_tax_post_discount,
      sum(sales_pre_tax_post_discount) rev_pre_tax_post_discount,
      sum(sales_pre_tax_pre_discount) rev_pre_tax_pre_discount,
      sum(cogs) cogs
   from
      base
      -- EXCLUDE low COGS items
   where
      cogs > 1
      -- EXCLUDE high cost, high price items
      and per_unit_price <= 500
      and per_unit_cost <= 500
   group by
      2,
      3,
      1
   order by
      2,
      3,
      1
)
,
monthly_location_rev_and_cogs_and_profit as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      rev_post_tax_post_discount - cogs profit_post_tax_post_discount,
      rev_pre_tax_post_discount - cogs profit_pre_tax_post_discount,
      rev_pre_tax_pre_discount - cogs profit_pre_tax_pre_discount,
   from
      monthly_location_rev_and_cogs
)
,
monthly_location_rev_and_cogs_and_profit_and_margin as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      profit_post_tax_post_discount,
      profit_pre_tax_post_discount,
      profit_pre_tax_pre_discount,
      -- margin
      profit_post_tax_post_discount / nullif(rev_post_tax_post_discount, 0) margin_post_tax_post_discount,
      profit_pre_tax_post_discount / nullif(rev_pre_tax_post_discount, 0) margin_pre_tax_post_discount,
      profit_pre_tax_pre_discount / nullif(rev_pre_tax_pre_discount, 0) margin_pre_tax_pre_discount
   from
      monthly_location_rev_and_cogs_and_profit
)
--     select * from monthly_location_rev_and_cogs_and_profit_and_margin
,
monthly_gm_analysis as
(
   select
      year_month,
      shopid,
      companyid,
      -- CHOOSE TO USE PRE TAX POST DISCOUNT
      rev_pre_tax_post_discount,
      lag(rev_pre_tax_post_discount) over (partition by shopid order by year_month) lag_rev_pre_tax_post_discount,
      cogs,
      profit_pre_tax_post_discount,
      margin_pre_tax_post_discount,
      count(distinct year_month) over (partition by shopid ) month_available,
      sum(cogs) over (partition by shopid order by year_month rows between 2 preceding and current row ) sum_cogs_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 1 preceding and current row ) avg_margin_2m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 2 preceding and current row ) avg_margin_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 5 preceding and current row ) avg_margin_6m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month ) avg_margin_all
   from
      monthly_location_rev_and_cogs_and_profit_and_margin
)
select
   *,
   (
      rev_pre_tax_post_discount - lag_rev_pre_tax_post_discount
   )
   / nullif(lag_rev_pre_tax_post_discount, 0) rev_change
from
   monthly_gm_analysis
   where shopid not in ("5e50449d6c0a910843356fa3",
                        "5e666af1aa809b085a98a646",
                        "56cf846ee38179985229e59e",
                        "5e6d0948b987950831b5092c",
                        "5ab806d6c182bb072e0b0e01",
                        "617b7da0e5cb65322b32781a",
                        "56cf84e4e38179985229e59f",
                        "5ee7c6a938fa9a08c8b7939c",
                        "5eebecd661ea0b08cdbcc386",
                        "56cf855ce38179985229e5a0",
                        "5ef12ad7bb5e7a08c5b3f4f3",
                        "5f1108355ae12008f4fef76e",
                        "5f123a5d12497908d2888dde",
                        "5f2309d579373608c93e6742",
                        "5f341ed02fdd5608e0821b45",
                        "5f8870200260fe08fe708db5",
                        "5f8870200260fe08fe708db6",
                        "5fac6a0b0643cc08efd1d327",
                        "56cf855ce38179985229e5a1",
                        "5d238c1955be2e07f5ee66e5",
                        "57b35a5d920f401e40146301",
                        "603d1a9eb9d7dc08cca27203",
                        "603d1a9eb9d7dc08cca27204",
                        "6046544f61e17808cff96fdb",
                        "6046544f61e17808cff96fdc",
                        "57b35aa8920f401e40146334",
                        "57b6b145920f4045cbe5fe81",
                        "6059219abe32aa08fa3a11dc",
                        "57b6b36a920f4046337f2618",
                        "58a7433f819d9e6ebd427617",
                        "60c0ffd6331cab296f02f49a",
                        "610c0a0d381ba0747c981ce9",
                        "611c0c98127cdd740cff94db",
                        "5a43f72b819d9e3ef9373675",
                        "611c0cc403d2e97dc2c159b0",
                        "5e6d0ac5b987950831b513e3",
                        "5a4563f1819d9e4ae6207e06",
                        "611c0ce7649819050ed4906b",
                        "611c0d02f572ec7882d64eea",
                        "611c0d26d5f1e86542fe3b8b",
                        "61672305b5e382719c7e4c2a",
                        "616dee0e4497e822dae13b21",
                        "616e5b13dd10fe4d3f34fd05",
                        "616e5b13dd10fe4d3f34fd06",
                        "619834e921c8017e23135674",
                        "5ab2d1a5819d9e21137cec3c",
                        "61bb746b82fddc50e058d711",
                        "61c3be757511d81170a29d51",
                        "61d49080eec1b51a3e33dc39",
                        "61d8912bdd71d40c2b68c5b3",
                        "61ba4efb9536472be6a8baa9",
                        "61d895fb3efbfb6da5210ae3",
                        "61d8aaf527205210191d7a1e",
                        "61e72d51073c4a79cfeccb62",
                        "61fdab9eae59d05c49422c3a",
                        "6202bd1c15425361f688aabf",
                        "6202bd1c15425361f688aac0",
                        "5b9f8e1db93307080d35b85f",
                        "5c2c250eabe12808989ac2f7",
                        "58be5603c182bb1f9bcd2ad5",
                        "5c8be64665a6a40808118621",
                        "606761051436a608e166db19",
                        "5ce2eb562d4dcf07fb7d0c5f",
                        "6176fa9eda707b1faba5f717",
                        "5b517867485da70830fcc04e",
                        "5b9f841a17b6e00812fe71f3",
                        "5da023586d8ce1082b9c91bd",
                        "5bbce3db420c21083300e410",
                        "5d238c1955be2e07f5ee66e4",
                        "5b209459062bd807b142a1f6",
                        "5cb1a5839fbb5f0818f6de55",
                        "5cda1adb183d6a0801347027",
                        "5be3903b79015e083194aa70",
                        "5d1ba174cdbd6d0803c99cfd",
                        "5a8a6eb8819d9e78641ab6ec",
                        "5db863825a04a2083b464083",
                        "5d8a7fb3a3274308311743d5",
                        "5d8a7fb3a3274308311743d6",
                        "5d9e425d52d15b0824a8a5d5",
                        "5dd6da52ce4015084c72e8db",
                        "5e07f51837cdd7082662f3c6",
                        "5e4d849981e3cd08326b6c14",
                        "5dfd2308fa462a086a8bfab8",
                        "5e0ee2710a7b7008346b06b4",
                        "5e4d849981e3cd08326b6c15",
                        "5dfd2308fa462a086a8bfab9",
                        "5e6d0adc12f2e208728f9369",
                        "5e6d0948b987950831b5092b",
                        "5ed589d96c0bf408dcc92d97",
                        "5eebecd761ea0b08cdbcc387",
                        "5ef12ad7bb5e7a08c5b3f4f2",
                        "5f63afd430bd7208ce1ab304",
                        "5f63afd430bd7208ce1ab305",
                        "5bf5ce54062bd807e436b473",
                        "5fac691a715f3808cd0aa82d",
                        "5d9d7fc27e2e46081e7e69f3",
                        "6034287d52503608c0b3ca48",
                        "5dfd40ea521ea90860c91848",
                        "6034287d52503608c0b3ca49",
                        "60636227ec491108d1c970d5",
                        "5a9a2274c182bb4ec00a72ca",
                        "60aea3bb1940943cecec2dbf",
                        "60a2f1610f75543c8e7dde50",
                        "5eb98f2d649a6508f87a6436",
                        "5e62a68447d90a086b8231c0",
                        "5e62a68447d90a086b8231c1",
                        "615b634cac08aa0a794158c4",
                        "60a2db8d0f75543c8e7d513b",
                        "60a2db8d0f75543c8e7d513c",
                        "60a4051e67406e7edad8ab88",
                        "60c0ff66082dcc61e42ad4a6",
                        "60c0ff66082dcc61e42ad4a7",
                        "6046c57057abe808d1f260fd",
                        "60636227ec491108d1c970d4",
                        "610abcaec3d09859f9b630d1",
                        "606cfaadba253e08c4189a63",
                        "607a321441cfa308f6e65b58",
                        "6197e31b21c8017e2310c10e",
                        "611d457db7c6f80914292394",
                        "610c0a0d381ba0747c981cea",
                        "610abcaec3d09859f9b630d2",
                        "611c0cb3a29e0f598f108a60",
                        "5fff453a81a63608d3a1a5b7",
                        "615237cc0937e52684b6c134",
                        "620b59c97852ca7a7dcc6f05",
                        "6196aa4deb62c87eba4014ad",
                        "619816781f88a1280f8a0c28",
                        "61d7b217a42c2f124d9b76b1",
                        "61cf4b452bc6657512b92d2a",
                        "61ebb060b04dc7510465f774",
                        "61eedaa0b882812f2f46a23d",
                        "61eedade6716dd2a2e0cfce5",
                        "5e6d0ae6e4a25a0834047467",
                        "5dc3315fe0faca07c6dfe849",
                        "5f7e17594445b008d2f63ebd",
                        "5f7f63053dbcad08cc9edbfb",
                        "5fff453a81a63608d3a1a5b8",
                        "603d5425a9ca7208f886d156",
                        "60e48dbbd5596d7c8a1d24dc",
                        "611c0d335901162ff90ef4dd",
                        "613f771aa96c863c00c1517d",
                        "6153b70fd343dc6230c4b67d",
                        "61699cdf032e1c1f920ec70f",
                        "61699cdf032e1c1f920ec710",
                        "61a7b4c6c8c7d85ea1846f14",
                        "61c390cfb863cb3baddb70f4",
                        "61d7b252ae999342ac05499e",
                        "61e6f857c5da34762f1c1fb5",
                        "61ebb00a335a531542342a32",
                        "620b59c97852ca7a7dcc6f04",
                        "6226640755810d3311e8ecd0")
""",engine)

In [4]:
blaze_gm_data['year_month'] = pd.to_datetime(blaze_gm_data['year_month'])

In [5]:
blaze_gm_data

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
0,2022-04-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,53.00,NaN,43.00,10.00,0.188679,1,43.00,0.188679,0.188679,0.188679,0.188679,NaN
1,2022-02-28,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,14.40,NaN,3.00,11.40,0.791667,2,3.00,0.791667,0.791667,0.791667,0.791667,NaN
2,2022-04-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,2906.35,14.40,4469.57,-1563.22,-0.537864,2,4472.57,0.126902,0.126902,0.126902,0.126902,200.829861
3,2021-08-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,1074911.92,NaN,509955.55,564956.37,0.525584,9,509955.55,0.525584,0.525584,0.525584,0.525584,NaN
4,2021-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,963022.94,1074911.92,467358.53,495664.41,0.514696,9,977314.08,0.520140,0.520140,0.520140,0.520140,-0.104091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,380092.77,298788.63,194186.06,185906.71,0.489109,7,429030.97,0.497369,0.499876,0.499876,0.499876,0.272113
63,2022-01-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,351261.17,380092.77,181632.45,169628.72,0.482913,7,523530.93,0.486011,0.492550,0.495636,0.495636,-0.075854
64,2022-02-28,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,313545.11,351261.17,164070.46,149474.65,0.476725,7,539888.97,0.479819,0.482916,0.491853,0.491853,-0.107373
65,2022-03-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,361236.93,313545.11,185504.42,175732.51,0.486474,7,531207.33,0.481599,0.482037,0.490957,0.490957,0.152105


In [6]:
blaze_gm_data[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,6
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,10
2022-03-31,8
2022-04-30,10


In [7]:
blaze_gm_data[blaze_gm_data['year_month'] == '2022-04-30'][['shopid','month_available']].groupby(['shopid']).mean()

,month_available
shopid,
5e7e9d34f795d008e7136fe1,9.0
5faf10dda4594c08c922b618,9.0
5fb5656cd5322c09020a0027,9.0
5fdce3b36c42c608cc9c8b8b,9.0
602c09ee9e113008ed666e63,9.0
607701db4683fe08ef9f2b44,5.0
609d85d3b6a049435d4faf02,5.0
60b913136fc90078bb683ee1,7.0
610daf318c9aff4915efe183,2.0


In [8]:
blaze_gm_data_long = blaze_gm_data[blaze_gm_data['month_available'] >= 5]

# 1m 

In [9]:
blaze_gm_data_long

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
3,2021-08-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,1074911.92,NaN,509955.55,564956.37,0.525584,9,509955.55,0.525584,0.525584,0.525584,0.525584,NaN
4,2021-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,963022.94,1074911.92,467358.53,495664.41,0.514696,9,977314.08,0.520140,0.520140,0.520140,0.520140,-0.104091
5,2021-10-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,1120273.17,963022.94,528530.24,591742.93,0.528213,9,1505844.32,0.521455,0.522831,0.522831,0.522831,0.163288
6,2021-11-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,1050567.37,1120273.17,484497.90,566069.47,0.538823,9,1480386.67,0.533518,0.527244,0.526829,0.526829,-0.062222
7,2021-12-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,1172783.78,1050567.37,536362.74,636421.04,0.542658,9,1549390.88,0.540741,0.536565,0.529995,0.529995,0.116334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,380092.77,298788.63,194186.06,185906.71,0.489109,7,429030.97,0.497369,0.499876,0.499876,0.499876,0.272113
63,2022-01-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,351261.17,380092.77,181632.45,169628.72,0.482913,7,523530.93,0.486011,0.492550,0.495636,0.495636,-0.075854
64,2022-02-28,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,313545.11,351261.17,164070.46,149474.65,0.476725,7,539888.97,0.479819,0.482916,0.491853,0.491853,-0.107373
65,2022-03-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,361236.93,313545.11,185504.42,175732.51,0.486474,7,531207.33,0.481599,0.482037,0.490957,0.490957,0.152105


In [10]:
# mom margin
gm_distr_mom = blaze_gm_data_long.groupby(['year_month'])['margin_pre_tax_post_discount'].describe().reset_index()
gm_distr_mom.index = pd.to_datetime(gm_distr_mom['year_month'])
gm_distr_mom

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.497450,0.011701,0.487583,0.488357,0.492444,0.504168,0.514696
2021-10-31,2021-10-31,6.0,0.498306,0.022464,0.460658,0.492747,0.499911,0.508064,0.528213
2021-11-30,2021-11-30,6.0,0.496347,0.025522,0.466641,0.479345,0.496106,0.504012,0.538823
2021-12-31,2021-12-31,8.0,0.502336,0.026345,0.473160,0.483350,0.494554,0.522181,0.542658
2022-01-31,2022-01-31,8.0,0.495824,0.052370,0.384271,0.482802,0.498250,0.530127,0.550789
2022-02-28,2022-02-28,8.0,0.494527,0.040478,0.417804,0.481473,0.486937,0.528965,0.542586
2022-03-31,2022-03-31,8.0,0.503612,0.031257,0.449393,0.486485,0.501254,0.524124,0.544807
2022-04-30,2022-04-30,8.0,0.499844,0.043616,0.410174,0.491674,0.501630,0.522211,0.554112


In [11]:
round(gm_distr_mom[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.52
dtype: float64

In [12]:
round(blaze_gm_data_long['margin_pre_tax_post_discount'].quantile(0.95),2)

0.54

# 2m

In [13]:
gm_distr_2m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_2m'].describe().reset_index()
gm_distr_2m.index = pd.to_datetime(gm_distr_2m['year_month'])
gm_distr_2m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500015,0.012293,0.487900,0.492876,0.498749,0.500410,0.520140
2021-10-31,2021-10-31,6.0,0.498498,0.015398,0.474120,0.494789,0.498416,0.503353,0.521455
2021-11-30,2021-11-30,6.0,0.497326,0.023375,0.463649,0.486774,0.498338,0.504981,0.533518
2021-12-31,2021-12-31,8.0,0.501897,0.025005,0.469901,0.483466,0.498684,0.514974,0.540741
2022-01-31,2022-01-31,8.0,0.499080,0.034802,0.442136,0.480756,0.491239,0.526976,0.543436
2022-02-28,2022-02-28,8.0,0.495175,0.046148,0.401038,0.482027,0.492593,0.529546,0.546687
2022-03-31,2022-03-31,8.0,0.499070,0.035239,0.433598,0.485365,0.493171,0.531387,0.540069
2022-04-30,2022-04-30,8.0,0.501728,0.037147,0.429784,0.489079,0.501442,0.523167,0.549459


In [14]:
round(gm_distr_2m[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.52
dtype: float64

In [15]:
round(blaze_gm_data_long['avg_margin_2m'].quantile(0.95),2)

0.54

## 3m

In [16]:
gm_distr_3m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_3m'].describe().reset_index()
gm_distr_3m.index = pd.to_datetime(gm_distr_3m['year_month'])
gm_distr_3m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500015,0.012293,0.487900,0.492876,0.498749,0.500410,0.520140
2021-10-31,2021-10-31,6.0,0.499987,0.013422,0.486052,0.490830,0.497952,0.504497,0.522831
2021-11-30,2021-11-30,6.0,0.497801,0.018543,0.471627,0.489641,0.497646,0.503665,0.527244
2021-12-31,2021-12-31,8.0,0.502241,0.023884,0.466820,0.486560,0.499938,0.515062,0.536565
2022-01-31,2022-01-31,8.0,0.497820,0.033892,0.442136,0.481977,0.490894,0.521218,0.542186
2022-02-28,2022-02-28,8.0,0.497562,0.036473,0.434025,0.482314,0.488222,0.527847,0.542320
2022-03-31,2022-03-31,8.0,0.497988,0.040515,0.417156,0.486161,0.493715,0.532551,0.541450
2022-04-30,2022-04-30,8.0,0.499328,0.037726,0.425790,0.487468,0.495991,0.528328,0.544750


In [17]:
round(gm_distr_3m[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.52
dtype: float64

In [18]:
round(blaze_gm_data_long['avg_margin_3m'].quantile(0.95),2)

0.54

## 6m

In [19]:
gm_distr_6m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_6m'].describe().reset_index()
gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
gm_distr_6m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500015,0.012293,0.487900,0.492876,0.498749,0.500410,0.520140
2021-10-31,2021-10-31,6.0,0.499987,0.013422,0.486052,0.490830,0.497952,0.504497,0.522831
2021-11-30,2021-11-30,6.0,0.499108,0.016372,0.481199,0.487959,0.497491,0.504514,0.526829
2021-12-31,2021-12-31,8.0,0.503021,0.019651,0.479592,0.488589,0.499938,0.511621,0.533584
2022-01-31,2022-01-31,8.0,0.497540,0.031215,0.442136,0.486467,0.492688,0.514706,0.542186
2022-02-28,2022-02-28,8.0,0.496199,0.034046,0.434025,0.485685,0.491111,0.516909,0.542320
2022-03-31,2022-03-31,8.0,0.497643,0.033532,0.437867,0.485566,0.491203,0.524041,0.539007
2022-04-30,2022-04-30,8.0,0.498165,0.035310,0.432328,0.485755,0.491265,0.526888,0.543324


In [20]:
round(gm_distr_6m[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.51
dtype: float64

In [21]:
round(blaze_gm_data_long['avg_margin_6m'].quantile(0.95),2)

0.54

## all time

In [22]:
gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])
round(gm_distr_all,3)

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.503,0.019,0.482,0.483,0.510,0.512,0.526
2021-09-30,2021-09-30,5.0,0.500,0.012,0.488,0.493,0.499,0.500,0.520
2021-10-31,2021-10-31,6.0,0.500,0.013,0.486,0.491,0.498,0.504,0.523
2021-11-30,2021-11-30,6.0,0.499,0.016,0.481,0.488,0.497,0.505,0.527
2021-12-31,2021-12-31,8.0,0.503,0.020,0.480,0.489,0.500,0.512,0.534
2022-01-31,2022-01-31,8.0,0.498,0.031,0.442,0.486,0.493,0.515,0.542
2022-02-28,2022-02-28,8.0,0.496,0.034,0.434,0.486,0.491,0.517,0.542
2022-03-31,2022-03-31,8.0,0.497,0.032,0.438,0.488,0.491,0.520,0.536
2022-04-30,2022-04-30,8.0,0.497,0.033,0.432,0.488,0.491,0.521,0.536


In [23]:
round(gm_distr_all[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.51
dtype: float64

In [24]:
round(blaze_gm_data_long['avg_margin_all'].quantile(0.95),2)

0.54

# Rev change

In [25]:
def gmv_change_variance_point_mapping(x):
    if x < -0.1:
        return -5
    elif x >= -0.1 and x <-0.05:
        return -2.5
    elif x >= -0.05 and x < 0:
        return 0
    elif x >= 0 and x < 0.05:
        return 5
    elif x >= 0.05 and x <= 0.5:
        return 10
    else:
        return numpy.nan
    

## blaze benchmark

In [ ]:
# rev_change_bm = blaze_gm_data_long.groupby(['year_month'])['rev_change'].describe().reset_index()
# rev_change_bm.index = pd.to_datetime(rev_change_bm['year_month'])
# rev_change_bm

In [ ]:
# rev_change_bm[['50%']].plot(kind = 'bar')

In [ ]:
# rev_change_bm[['year_month','50%']]

In [ ]:
# rev_change_bm_with_weight = rev_change_bm[['year_month','50%']]
# rev_change_bm_with_weight['weight'] = [0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
# rev_change_bm_with_weight.columns = ['year_month','rev_change_bm','rev_change_weight']
# rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
# rev_change_bm_with_weight

## state website published benchmark

In [26]:
# use own bm

rev_change_bm_with_weight = pd.DataFrame([numpy.nan,-0.07, 0.01, -0.07, 0.09, -0.11, 0.02, 0.05,numpy.nan])
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values
rev_change_bm_with_weight['weight'] = [0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
rev_change_bm_with_weight.columns = ['rev_change_bm','year_month','rev_change_weight']
rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
rev_change_bm_with_weight

,rev_change_bm,year_month,rev_change_weight
0,NaN,2021-08-31,0.36
1,-0.07,2021-09-30,0.44
2,0.01,2021-10-31,0.52
3,-0.07,2021-11-30,0.60
4,0.09,2021-12-31,0.68
5,-0.11,2022-01-31,0.76
6,0.02,2022-02-28,0.84
7,0.05,2022-03-31,0.92
8,NaN,2022-04-30,1.00


In [27]:

blaze_gm_data_long_with_rev  = pd.merge(blaze_gm_data_long,rev_change_bm_with_weight,on=['year_month'], how='inner')
blaze_gm_data_long_with_rev['rev_change_var'] = blaze_gm_data_long_with_rev['rev_change'] - blaze_gm_data_long_with_rev['rev_change_bm']
blaze_gm_data_long_with_rev['rev_change_points'] = [gmv_change_variance_point_mapping(n) for n in blaze_gm_data_long_with_rev['rev_change_var']]
blaze_gm_data_long_with_rev['rev_change_total'] = blaze_gm_data_long_with_rev['rev_change_points'] * blaze_gm_data_long_with_rev['rev_change_weight']



In [28]:
blaze_gm_data_long_with_rev.tail()

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
57,2022-04-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,328152.94,357414.71,150972.39,177180.55,0.539933,9,466235.45,0.540263,0.535790,0.525057,0.517809,-0.081871,NaN,1.0,NaN,NaN,NaN
58,2022-04-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,933645.54,920297.42,465074.48,468571.06,0.501873,9,1324097.81,0.500402,0.495976,0.492606,0.491819,0.014504,NaN,1.0,NaN,NaN,NaN
59,2022-04-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,112075.05,111940.89,66104.77,45970.28,0.410174,5,168100.92,0.429784,0.425790,0.432328,0.432328,0.001198,NaN,1.0,NaN,NaN,NaN
60,2022-04-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,523228.12,554036.01,260888.19,262339.93,0.501387,9,786014.80,0.502481,0.496006,0.489923,0.490812,-0.055606,NaN,1.0,NaN,NaN,NaN
61,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,384031.05,361236.93,200041.88,183989.17,0.479100,7,549616.76,0.482787,0.480766,0.486658,0.489263,0.063100,NaN,1.0,NaN,NaN,NaN


# inventory

In [84]:
blaze_inv_data = pd.read_sql_query("""
with base as (
    select *,
    last_day(date) year_month,
    current_quantity * unit_cost total_value
    from blaze_data.inventory
    where  lower(product_category) not like '%sample%'
    and lower(product_category) not like '%display%'
    and lower(product_category) not like '%promo%'
    and lower(product_category) not like '%test%'
    -- EXCLUDE SMALL COGS
    and unit_cost > 0.01
    and current_quantity > 0
    and current_quantity < 1000
    and date in ('2021-08-29','2021-09-26','2021-10-31','2021-11-28','2021-12-26','2022-01-30','2022-02-27','2022-03-27','2022-04-24')
    and state = 'MA'
)

select
year_month,
shop_id shopid,
company_id companyid,
sum(total_value) inventory
from base
group by 1,2,3
order by 2,3,1


""",engine)

In [85]:
blaze_inv_data

,year_month,shopid,companyid,inventory
0,2022-04-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,366966.439513
1,2022-04-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,368067.216667
2,2022-04-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,363792.306905
3,2022-04-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,251101.146667
4,2022-04-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,356249.873129
5,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,375386.115000
6,2022-04-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,221189.905952
7,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,414537.317500
8,2022-04-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,36901.690000
9,2022-04-30,61cb592938b1e14beb10bb7a,61cb592938b1e14beb10bb61,270503.570833


In [58]:
blaze_inv_data['year_month'] = pd.to_datetime(blaze_inv_data['year_month'])

In [59]:
set(blaze_gm_data['shopid']) - set(blaze_inv_data['shopid'])

{'605cef20813d4c08ebf50c06'}

# joined

In [60]:
data_joined = pd.merge(blaze_gm_data_long_with_rev,blaze_inv_data,on=['year_month','shopid','companyid'], how='inner')


In [61]:
data_joined

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory
0,2022-04-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,882059.08,898184.14,393299.42,488759.66,0.554112,9,1265995.08,0.549459,0.544750,0.543324,0.536493,-0.017953,NaN,1.0,NaN,NaN,NaN,366966.439513
1,2022-04-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,386938.01,342267.56,195068.84,191869.17,0.495865,9,512266.12,0.491177,0.489702,0.483045,0.484048,0.130513,NaN,1.0,NaN,NaN,NaN,368067.216667
2,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,362430.54,351007.09,175306.13,187124.41,0.516304,5,484698.55,0.517469,0.525841,0.532379,0.532379,0.032545,NaN,1.0,NaN,NaN,NaN,375386.115000
3,2022-04-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,328152.94,357414.71,150972.39,177180.55,0.539933,9,466235.45,0.540263,0.535790,0.525057,0.517809,-0.081871,NaN,1.0,NaN,NaN,NaN,251101.146667
4,2022-04-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,933645.54,920297.42,465074.48,468571.06,0.501873,9,1324097.81,0.500402,0.495976,0.492606,0.491819,0.014504,NaN,1.0,NaN,NaN,NaN,363792.306905
5,2022-04-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,112075.05,111940.89,66104.77,45970.28,0.410174,5,168100.92,0.429784,0.425790,0.432328,0.432328,0.001198,NaN,1.0,NaN,NaN,NaN,221189.905952
6,2022-04-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,523228.12,554036.01,260888.19,262339.93,0.501387,9,786014.80,0.502481,0.496006,0.489923,0.490812,-0.055606,NaN,1.0,NaN,NaN,NaN,356249.873129
7,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,384031.05,361236.93,200041.88,183989.17,0.479100,7,549616.76,0.482787,0.480766,0.486658,0.489263,0.063100,NaN,1.0,NaN,NaN,NaN,414537.317500


In [62]:
data_joined['inventory_turnover'] = data_joined['sum_cogs_3m'] / data_joined['inventory'] * 4
data_joined['year_month'] = pd.to_datetime(data_joined['year_month'])

In [63]:
data_joined[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-04-30,8


In [64]:
data_joined[data_joined['year_month'] >= '2021-10-31'].groupby(['year_month'])['inventory_turnover'].describe().reset_index()

,year_month,count,mean,std,min,25%,50%,75%,max
0,2022-04-30,8.0,7.960768,4.20003,3.039938,5.268768,6.497074,10.068968,14.558832


In [65]:
round(data_joined['inventory_turnover'].quantile(0.95),2)

14.29

In [67]:
# data_joined_week = pd.merge(blaze_gm_data_long_with_rev,blaze_inv_data,on=['shopid','companyid'], how='inner')

# data_joined_week['inventory_turnover'] = data_joined_week['sum_cogs_3m'] / data_joined_week['inventory'] * 4
# data_joined_week['year_month'] = pd.to_datetime(data_joined_week['year_month'])

# data_joined_week

# UW

In [68]:
recent = data_joined[data_joined['year_month'] == '2022-04-30']
recent = recent.reset_index(drop = True)

In [69]:
recent[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-04-30,8


In [70]:
recent

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory,inventory_turnover
0,2022-04-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,882059.08,898184.14,393299.42,488759.66,0.554112,9,1265995.08,0.549459,0.544750,0.543324,0.536493,-0.017953,NaN,1.0,NaN,NaN,NaN,366966.439513,13.799573
1,2022-04-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,386938.01,342267.56,195068.84,191869.17,0.495865,9,512266.12,0.491177,0.489702,0.483045,0.484048,0.130513,NaN,1.0,NaN,NaN,NaN,368067.216667,5.567093
2,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,362430.54,351007.09,175306.13,187124.41,0.516304,5,484698.55,0.517469,0.525841,0.532379,0.532379,0.032545,NaN,1.0,NaN,NaN,NaN,375386.115000,5.164800
3,2022-04-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,328152.94,357414.71,150972.39,177180.55,0.539933,9,466235.45,0.540263,0.535790,0.525057,0.517809,-0.081871,NaN,1.0,NaN,NaN,NaN,251101.146667,7.427054
4,2022-04-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,933645.54,920297.42,465074.48,468571.06,0.501873,9,1324097.81,0.500402,0.495976,0.492606,0.491819,0.014504,NaN,1.0,NaN,NaN,NaN,363792.306905,14.558832
5,2022-04-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,112075.05,111940.89,66104.77,45970.28,0.410174,5,168100.92,0.429784,0.425790,0.432328,0.432328,0.001198,NaN,1.0,NaN,NaN,NaN,221189.905952,3.039938
6,2022-04-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,523228.12,554036.01,260888.19,262339.93,0.501387,9,786014.80,0.502481,0.496006,0.489923,0.490812,-0.055606,NaN,1.0,NaN,NaN,NaN,356249.873129,8.825432
7,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,384031.05,361236.93,200041.88,183989.17,0.479100,7,549616.76,0.482787,0.480766,0.486658,0.489263,0.063100,NaN,1.0,NaN,NaN,NaN,414537.317500,5.303424


In [71]:
# recent_week = data_joined_week[data_joined_week['year_month'] == '2022-04-30']
# recent_week = recent_week.reset_index(drop = True)

# recent_week[['shopid','inventory_turnover']].groupby(['shopid']).describe()

## gm % score

In [72]:
def get_gm_perc_scores(threshold,gm_3,gm_6,gm_all):
    # 3m score
    if gm_3 <= threshold[0][0]:
        score_3m = -2
    elif threshold[0][0] < gm_3 <= threshold[0][1]:
        score_3m = 0
    elif threshold[0][1] < gm_3 <= threshold[0][2]:
        score_3m = 5
    elif threshold[0][2] < gm_3 <= threshold[0][3]:
        score_3m = 5
    else:
        score_3m = 0
    # 6m score
    if gm_6 <= threshold[1][0]:
        score_6m = -2
    elif threshold[1][0] < gm_6 <= threshold[1][1]:
        score_6m = 0
    elif threshold[1][1] < gm_6 <= threshold[1][2]:
        score_6m = 5
    elif threshold[1][2] < gm_6 <= threshold[1][3]:
        score_6m = 5
    else:
        score_6m = 0
    # all time score
    if gm_all <= threshold[2][0]:
        score_all = -2
    elif threshold[2][0] < gm_all <= threshold[2][1]:
        score_all = 0
    elif threshold[2][1] < gm_all <= threshold[2][2]:
        score_all = 5
    elif threshold[2][2] < gm_all <= threshold[2][3]:
        score_all = 5
    else:
        score_all = 0
    return score_3m,score_6m,score_all
    

In [73]:
gm_threshold = [[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7]]

In [ ]:
#gm_threshold = [[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7]]

In [ ]:
#gm_threshold = [[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7]]

In [ ]:
#gm_threshold = [[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7]]

In [74]:
# use 3,6,9m
gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['avg_margin_3m'],row['avg_margin_6m'],row['avg_margin_all']),axis = 1)
recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



In [ ]:
# # use 1,2,3m
# gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['margin_pre_tax_post_discount'],row['avg_margin_2m'],row['avg_margin_3m']),axis = 1)
# recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_1m_score', 'gm_2m_score','gm_3m_score'])],axis=1)



In [ ]:
recent_with_gm

## inventory turnover score

In [75]:
recent_with_gm['turnover_score'] = [10 if (recent_with_gm['inventory_turnover'][i] >= 6 and recent_with_gm['inventory_turnover'][i] <= 15) else 0 for i in range(len(recent_with_gm))]



In [76]:
recent_with_gm

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score
0,2022-04-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,882059.08,898184.14,393299.42,488759.66,0.554112,9,1265995.08,0.549459,0.544750,0.543324,0.536493,-0.017953,NaN,1.0,NaN,NaN,NaN,366966.439513,13.799573,5,5,5,10
1,2022-04-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,386938.01,342267.56,195068.84,191869.17,0.495865,9,512266.12,0.491177,0.489702,0.483045,0.484048,0.130513,NaN,1.0,NaN,NaN,NaN,368067.216667,5.567093,-2,-2,-2,0
2,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,362430.54,351007.09,175306.13,187124.41,0.516304,5,484698.55,0.517469,0.525841,0.532379,0.532379,0.032545,NaN,1.0,NaN,NaN,NaN,375386.115000,5.164800,5,5,5,0
3,2022-04-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,328152.94,357414.71,150972.39,177180.55,0.539933,9,466235.45,0.540263,0.535790,0.525057,0.517809,-0.081871,NaN,1.0,NaN,NaN,NaN,251101.146667,7.427054,5,5,5,10
4,2022-04-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,933645.54,920297.42,465074.48,468571.06,0.501873,9,1324097.81,0.500402,0.495976,0.492606,0.491819,0.014504,NaN,1.0,NaN,NaN,NaN,363792.306905,14.558832,0,0,0,10
5,2022-04-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,112075.05,111940.89,66104.77,45970.28,0.410174,5,168100.92,0.429784,0.425790,0.432328,0.432328,0.001198,NaN,1.0,NaN,NaN,NaN,221189.905952,3.039938,-2,-2,-2,0
6,2022-04-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,523228.12,554036.01,260888.19,262339.93,0.501387,9,786014.80,0.502481,0.496006,0.489923,0.490812,-0.055606,NaN,1.0,NaN,NaN,NaN,356249.873129,8.825432,0,-2,0,10
7,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,384031.05,361236.93,200041.88,183989.17,0.479100,7,549616.76,0.482787,0.480766,0.486658,0.489263,0.063100,NaN,1.0,NaN,NaN,NaN,414537.317500,5.303424,-2,-2,-2,0


## rev change score

In [81]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['shopid'] == '60b913136fc90078bb683ee1']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
15,2021-10-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,175986.61,NaN,87132.49,88854.12,0.504891,7,87132.49,0.504891,0.504891,0.504891,0.504891,NaN,0.01,0.52,NaN,NaN,NaN
21,2021-11-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,298788.63,175986.61,147712.42,151076.21,0.505629,7,234844.91,0.505260,0.505260,0.505260,0.505260,0.697792,-0.07,0.60,0.767792,NaN,NaN
29,2021-12-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,380092.77,298788.63,194186.06,185906.71,0.489109,7,429030.97,0.497369,0.499876,0.499876,0.499876,0.272113,0.09,0.68,0.182113,10.0,6.8
37,2022-01-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,351261.17,380092.77,181632.45,169628.72,0.482913,7,523530.93,0.486011,0.492550,0.495636,0.495636,-0.075854,-0.11,0.76,0.034146,5.0,3.8
45,2022-02-28,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,313545.11,351261.17,164070.46,149474.65,0.476725,7,539888.97,0.479819,0.482916,0.491853,0.491853,-0.107373,0.02,0.84,-0.127373,-5.0,-4.2
53,2022-03-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,361236.93,313545.11,185504.42,175732.51,0.486474,7,531207.33,0.481599,0.482037,0.490957,0.490957,0.152105,0.05,0.92,0.102105,10.0,9.2
61,2022-04-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,384031.05,361236.93,200041.88,183989.17,0.479100,7,549616.76,0.482787,0.480766,0.486658,0.489263,0.063100,NaN,1.00,NaN,NaN,NaN


In [77]:
rev_change_score = blaze_gm_data_long_with_rev[['shopid','rev_change_total']].groupby('shopid').sum().reset_index()

In [78]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
rev_change_score.sort_values(by = 'rev_change_total')


,shopid,rev_change_total,rev_change_score
6,609d85d3b6a049435d4faf02,0.0,0.0
0,5e7e9d34f795d008e7136fe1,5.1,5.1
5,607701db4683fe08ef9f2b44,13.4,10.0
7,60b913136fc90078bb683ee1,15.6,10.0
4,602c09ee9e113008ed666e63,21.6,10.0
3,5fdce3b36c42c608cc9c8b8b,22.4,10.0
1,5faf10dda4594c08c922b618,36.2,10.0
2,5fb5656cd5322c09020a0027,39.1,10.0


## gm $ score

In [ ]:
gm_dollar_score = blaze_gm_data_long_with_rev[['shopid','profit_pre_tax_post_discount']].groupby('shopid').mean().reset_index()

In [ ]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['profit_pre_tax_post_discount'][i] >= 200000 else 0 for i in range(len(rev_change_score))]



In [ ]:
recent_with_gm_and_rev_change = pd.merge(recent_with_gm,rev_change_score[['shopid','rev_change_score']],on=['shopid'], how='inner')
recent_with_gm_and_rev_change

In [ ]:
final = pd.merge(recent_with_gm_and_rev_change,gm_dollar_score[['shopid','gm_dollar_score']],on=['shopid'], how='inner')



In [ ]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit'] = [round(min(final['sum_cogs_3m'][i],final['inventory'][i]),0) for i in range(len(final))]
                        



In [ ]:
# final['total_gm_perc_score'] = final['gm_1m_score'] + final['gm_2m_score'] + final['gm_3m_score']
# final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
# final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
# final['credit_limit'] = [round(min(final['sum_cogs_3m'][i],final['inventory'][i]),0) for i in range(len(final))]
                        



In [ ]:
final

# Proposal Analysis

In [ ]:
final['total_score'].describe()

In [ ]:
pass_list = final[final['total_score'] >= 24]

In [ ]:
final_pass = pass_list[(pass_list['profit_pre_tax_post_discount'] >= 25000) & (pass_list['credit_limit'] >= 30000)]
final_pass = final_pass.reset_index(drop = True)

In [ ]:
final_pass

In [ ]:
def calculate_interest_rate(score, full_score):
    score_ratio = score / full_score
    placeholder = (1 + 0.5 * (1 - score_ratio)) * 0.015
    rate = placeholder * 12
    return round(placeholder, 4), round(rate, 4)

In [ ]:
final_pass['annual_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[1] for i in range(len(final_pass))]
final_pass['monthly_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[0] for i in range(len(final_pass))]



In [ ]:
final_pass

In [ ]:
final_pass['credit_limit_final'] = [min(max(final_pass['credit_limit'][i],50000),250000) for i in range(len(final_pass))]

In [ ]:
#final_pass.to_csv('blaze_final_0524_369.csv')

# ~ END ~

In [ ]:
# ['5e7e9d34f795d008e7136fe1', '602c09ee9e113008ed666e63',
#        '5fb5656cd5322c09020a0027', '5fdce3b36c42c608cc9c8b8b',
#        '607701db4683fe08ef9f2b44']

In [ ]:
final_pass

In [ ]:
final_pass['credit_limit'].describe()

In [ ]:
final_pass['credit_limit'].sum()

In [ ]:
final_pass['credit_limit_final'].describe()

In [ ]:
final_pass['credit_limit_final'].sum()

In [ ]:
final_pass.shape[0]/335

In [ ]:
final_pass.shape[0] / blaze_gm_data.shape[0]

In [ ]:
blaze_gm_data['shopid'].nunique()

In [ ]:
final_pass['rev_pre_tax_post_discount'].describe()

In [ ]:
final_pass

In [ ]:
#final_pass.to_csv('final_pass_0421_1.csv')

# wow rev change

In [ ]:
#you can also query like this
wow = pd.read_sql_query("""
with base as
(
   select
      *,
      --last_day(week_start) year_month,
      retail_value / quantity_sold per_unit_price,
      cogs / quantity_sold per_unit_cost,
      sales.retail_value - sales.total_discount + sales.total_tax sales_post_tax_post_discount,
      sales.retail_value - sales.total_discount sales_pre_tax_post_discount,
      sales.retail_value sales_pre_tax_pre_discount,
   from
      blaze_data.sales sales
      -- EXCLUDE SAMPLES,DISPLAY,PROMO and test
   where
      lower(product_category) not like '%sample%'
      and lower(product_category) not like '%display%'
      and lower(product_category) not like '%promo%'
     and lower(product_category) not like '%test%'
)


,
monthly_location_rev_and_cogs as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      sum(sales_post_tax_post_discount) rev_post_tax_post_discount,
      sum(sales_pre_tax_post_discount) rev_pre_tax_post_discount,
      sum(sales_pre_tax_pre_discount) rev_pre_tax_pre_discount,
      sum(cogs) cogs
   from
      base
      -- EXCLUDE low COGS items
   where
      cogs > 1
      -- EXCLUDE high cost, high price items
      and per_unit_price <= 500
      and per_unit_cost <= 500
   group by
      3,
      4,
      1,
      2
   order by
      3,
      4,
      1,
      2
)
,
monthly_location_rev_and_cogs_and_profit as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      rev_post_tax_post_discount - cogs profit_post_tax_post_discount,
      rev_pre_tax_post_discount - cogs profit_pre_tax_post_discount,
      rev_pre_tax_pre_discount - cogs profit_pre_tax_pre_discount,
   from
      monthly_location_rev_and_cogs
)
,
monthly_location_rev_and_cogs_and_profit_and_margin as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      profit_post_tax_post_discount,
      profit_pre_tax_post_discount,
      profit_pre_tax_pre_discount,
      -- margin
      profit_post_tax_post_discount / nullif(rev_post_tax_post_discount, 0) margin_post_tax_post_discount,
      profit_pre_tax_post_discount / nullif(rev_pre_tax_post_discount, 0) margin_pre_tax_post_discount,
      profit_pre_tax_pre_discount / nullif(rev_pre_tax_pre_discount, 0) margin_pre_tax_pre_discount
   from
      monthly_location_rev_and_cogs_and_profit
)
,
monthly_gm_analysis as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- CHOOSE TO USE PRE TAX POST DISCOUNT
      rev_pre_tax_post_discount,
      lag(rev_pre_tax_post_discount) over (partition by shopid order by week_start) lag_rev_pre_tax_post_discount,
      cogs,
      profit_pre_tax_post_discount,
      margin_pre_tax_post_discount,
      count(distinct week_start) over (partition by shopid ) month_available
    --  sum(cogs) over (partition by shopid order by week_start rows between 2 preceding and current row ) sum_cogs_3m,
     -- avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start rows between 2 preceding and current row ) avg_margin_3m,
     -- avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start rows between 5 preceding and current row ) avg_margin_6m,
      --avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start ) avg_margin_all
   from
      monthly_location_rev_and_cogs_and_profit_and_margin
)
select
   *,
   (
      rev_pre_tax_post_discount - lag_rev_pre_tax_post_discount
   )
   / nullif(lag_rev_pre_tax_post_discount, 0) rev_change
from
   monthly_gm_analysis
where shopid not in ("5e50449d6c0a910843356fa3",
"5e666af1aa809b085a98a646",
"56cf846ee38179985229e59e",
"5e6d0948b987950831b5092c",
"5ab806d6c182bb072e0b0e01",
"617b7da0e5cb65322b32781a",
"56cf84e4e38179985229e59f",
"5ee7c6a938fa9a08c8b7939c",
"5eebecd661ea0b08cdbcc386",
"56cf855ce38179985229e5a0",
"5ef12ad7bb5e7a08c5b3f4f3",
"5f1108355ae12008f4fef76e",
"5f123a5d12497908d2888dde",
"5f2309d579373608c93e6742",
"5f341ed02fdd5608e0821b45",
"5f8870200260fe08fe708db5",
"5f8870200260fe08fe708db6",
"5fac6a0b0643cc08efd1d327",
"56cf855ce38179985229e5a1",
"5d238c1955be2e07f5ee66e5",
"57b35a5d920f401e40146301",
"603d1a9eb9d7dc08cca27203",
"603d1a9eb9d7dc08cca27204",
"6046544f61e17808cff96fdb",
"6046544f61e17808cff96fdc",
"57b35aa8920f401e40146334",
"57b6b145920f4045cbe5fe81",
"6059219abe32aa08fa3a11dc",
"57b6b36a920f4046337f2618",
"58a7433f819d9e6ebd427617",
"60c0ffd6331cab296f02f49a",
"610c0a0d381ba0747c981ce9",
"611c0c98127cdd740cff94db",
"5a43f72b819d9e3ef9373675",
"611c0cc403d2e97dc2c159b0",
"5e6d0ac5b987950831b513e3",
"5a4563f1819d9e4ae6207e06",
"611c0ce7649819050ed4906b",
"611c0d02f572ec7882d64eea",
"611c0d26d5f1e86542fe3b8b",
"61672305b5e382719c7e4c2a",
"616dee0e4497e822dae13b21",
"616e5b13dd10fe4d3f34fd05",
"616e5b13dd10fe4d3f34fd06",
"619834e921c8017e23135674",
"5ab2d1a5819d9e21137cec3c",
"61bb746b82fddc50e058d711",
"61c3be757511d81170a29d51",
"61d49080eec1b51a3e33dc39",
"61d8912bdd71d40c2b68c5b3",
"61ba4efb9536472be6a8baa9",
"61d895fb3efbfb6da5210ae3",
"61d8aaf527205210191d7a1e",
"61e72d51073c4a79cfeccb62",
"61fdab9eae59d05c49422c3a",
"6202bd1c15425361f688aabf",
"6202bd1c15425361f688aac0",
"5b9f8e1db93307080d35b85f",
"5c2c250eabe12808989ac2f7",
"58be5603c182bb1f9bcd2ad5",
"5c8be64665a6a40808118621",
"606761051436a608e166db19",
"5ce2eb562d4dcf07fb7d0c5f",
"6176fa9eda707b1faba5f717",
"5b517867485da70830fcc04e",
"5b9f841a17b6e00812fe71f3",
"5da023586d8ce1082b9c91bd",
"5bbce3db420c21083300e410",
"5d238c1955be2e07f5ee66e4",
"5b209459062bd807b142a1f6",
"5cb1a5839fbb5f0818f6de55",
"5cda1adb183d6a0801347027",
"5be3903b79015e083194aa70",
"5d1ba174cdbd6d0803c99cfd",
"5a8a6eb8819d9e78641ab6ec",
"5db863825a04a2083b464083",
"5d8a7fb3a3274308311743d5",
"5d8a7fb3a3274308311743d6",
"5d9e425d52d15b0824a8a5d5",
"5dd6da52ce4015084c72e8db",
"5e07f51837cdd7082662f3c6",
"5e4d849981e3cd08326b6c14",
"5dfd2308fa462a086a8bfab8",
"5e0ee2710a7b7008346b06b4",
"5e4d849981e3cd08326b6c15",
"5dfd2308fa462a086a8bfab9",
"5e6d0adc12f2e208728f9369",
"5e6d0948b987950831b5092b",
"5ed589d96c0bf408dcc92d97",
"5eebecd761ea0b08cdbcc387",
"5ef12ad7bb5e7a08c5b3f4f2",
"5f63afd430bd7208ce1ab304",
"5f63afd430bd7208ce1ab305",
"5bf5ce54062bd807e436b473",
"5fac691a715f3808cd0aa82d",
"5d9d7fc27e2e46081e7e69f3",
"6034287d52503608c0b3ca48",
"5dfd40ea521ea90860c91848",
"6034287d52503608c0b3ca49",
"60636227ec491108d1c970d5",
"5a9a2274c182bb4ec00a72ca",
"60aea3bb1940943cecec2dbf",
"60a2f1610f75543c8e7dde50",
"5eb98f2d649a6508f87a6436",
"5e62a68447d90a086b8231c0",
"5e62a68447d90a086b8231c1",
"615b634cac08aa0a794158c4",
"60a2db8d0f75543c8e7d513b",
"60a2db8d0f75543c8e7d513c",
"60a4051e67406e7edad8ab88",
"60c0ff66082dcc61e42ad4a6",
"60c0ff66082dcc61e42ad4a7",
"6046c57057abe808d1f260fd",
"60636227ec491108d1c970d4",
"610abcaec3d09859f9b630d1",
"606cfaadba253e08c4189a63",
"607a321441cfa308f6e65b58",
"6197e31b21c8017e2310c10e",
"611d457db7c6f80914292394",
"610c0a0d381ba0747c981cea",
"610abcaec3d09859f9b630d2",
"611c0cb3a29e0f598f108a60",
"5fff453a81a63608d3a1a5b7",
"615237cc0937e52684b6c134",
"620b59c97852ca7a7dcc6f05",
"6196aa4deb62c87eba4014ad",
"619816781f88a1280f8a0c28",
"61d7b217a42c2f124d9b76b1",
"61cf4b452bc6657512b92d2a",
"61ebb060b04dc7510465f774",
"61eedaa0b882812f2f46a23d",
"61eedade6716dd2a2e0cfce5",
"5e6d0ae6e4a25a0834047467",
"5dc3315fe0faca07c6dfe849",
"5f7e17594445b008d2f63ebd",
"5f7f63053dbcad08cc9edbfb",
"5fff453a81a63608d3a1a5b8",
"603d5425a9ca7208f886d156",
"60e48dbbd5596d7c8a1d24dc",
"611c0d335901162ff90ef4dd",
"613f771aa96c863c00c1517d",
"6153b70fd343dc6230c4b67d",
"61699cdf032e1c1f920ec70f",
"61699cdf032e1c1f920ec710",
"61a7b4c6c8c7d85ea1846f14",
"61c390cfb863cb3baddb70f4",
"61d7b252ae999342ac05499e",
"61e6f857c5da34762f1c1fb5",
"61ebb00a335a531542342a32",
"620b59c97852ca7a7dcc6f04",
"6226640755810d3311e8ecd0")
""",engine)

In [ ]:
wow

In [ ]:
wow_bm = wow.groupby(['week_start','week_end'])['rev_change'].describe().reset_index()
wow_bm.index = pd.to_datetime(wow_bm['week_start'])
wow_bm

In [ ]:
wow_bm[['50%']].plot(kind= 'bar',figsize = (10,8))